In [1]:
import numpy_financial as npf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from ipywidgets import interact


plt.rcParams['font.size'] = '20'

In [2]:
mortgage_interest = widgets.FloatSlider(
    value=3.25,
    min=2,
    max=5,
    step=0.05,
    description='Mortgage Interest in %',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

home_appreciation_rate = widgets.FloatSlider(
    value=2,
    min=-4,
    max=6,
    step=0.1,
    description='Home Appreciation %, Annual',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

rent_increase_per_year = widgets.FloatSlider(
    value=2,
    min=0,
    max=6,
    step=0.1,
    description='Rent Increase %, Annual',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)


rental_income_monthly = widgets.BoundedIntText(
    value=0,
    min=0,
    max=5000,
    step=100,
    description='Monthly Rental Income',
    disabled=False
)

first_month_taxes_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=5000,
    step=10,
    description='Taxes',
    disabled=False
)

first_month_insurance_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=5000,
    step=10,
    description='Insurance',
    disabled=False
)

first_month_maintenance_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=5000,
    step=10,
    description='Maintenance',
    disabled=False
)

closing_costs_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100000,
    step=10,
    description='Closing Costs',
    disabled=False
)

home_value = widgets.BoundedIntText(
    value=750000,
    min=200000,
    max=1500000,
    step=1000,
    description='Home Price',
    disabled=False
)

down_payment_pct = widgets.IntSlider(
    value=20,
    min=0,
    max=30,
    step=1,
    description='Down Payment %',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

tax_rate_pct_widget = widgets.IntSlider(
    value=20,
    min=0,
    max=60,
    step=1,
    description='Income Tax %',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

mortgage_years = widgets.IntSlider(
    value=30,
    min=10,
    max=30,
    step=1,
    description='Mortgage Years',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

tax_adjust = widgets.ToggleButton(
    value=False,
    description='Tax Adjust',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Adjust deductible things by the income tax rate',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

def plot_payments(df, title="Monthly Mortgage Payment"):
    ax = df.plot(x="month", kind="bar", stacked=True, figsize=(27, 7), title=title)
    plt.xticks(rotation=90)
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(12))
    ax.set_yticklabels(["{:,}".format(int(x)) for x in ax.get_yticks().tolist()])
    plt.show()




In [4]:

@interact
def derive_df(
    mortgage_years=mortgage_years,
    home_value=home_value, 
    down_payment_pct=down_payment_pct, 
    mortgage_interest_pct=mortgage_interest, 
    closing_costs=closing_costs_widget,
    home_appreciation_rate_pct=home_appreciation_rate,
    rent_increase_per_year_pct=rent_increase_per_year,
    first_month_taxes=first_month_taxes_widget,
    first_month_insurance=first_month_insurance_widget,
    first_month_maintenance=first_month_maintenance_widget,
    rental_income_monthly=rental_income_monthly,
    tax_rate_pct=tax_rate_pct_widget,
    tax_adjust=tax_adjust,
):
    closing_costs = -closing_costs
    first_month_maintenance = -first_month_maintenance
    first_month_taxes = -first_month_taxes
    first_month_insurance = -first_month_insurance
    
    periods = np.arange(mortgage_years*12)
    down_payment = home_value * down_payment_pct/100
    mortgage_amount = home_value - down_payment
    
    interest_payment = npf.ipmt(mortgage_interest_pct / 100 / 12, periods, 12 * mortgage_years, mortgage_amount)
    principal_payment = npf.ppmt(mortgage_interest_pct / 100 / 12, periods, 12 * mortgage_years, mortgage_amount)
    total_payment = npf.pmt(mortgage_interest_pct / 12, 12 * mortgage_years, mortgage_amount)
  
    df = pd.DataFrame(
    data={
        "month": periods,
        "interest": interest_payment,
        "principal": principal_payment,
        "home_value":home_value * np.exp(periods * home_appreciation_rate_pct/100/12),  
        "rental_income":rental_income_monthly * np.exp(periods * rent_increase_per_year_pct/100/12), 
        }
    )

    df["taxes"] = first_month_taxes/home_value*df['home_value']  
    df["insurance"] = first_month_insurance/home_value*df['home_value']  
    df["closing_costs"] =  np.zeros(360)
    df.at[0, 'closing_costs'] = closing_costs
    df["maintenance"] =  first_month_maintenance/home_value*df['home_value']
    if tax_adjust:
        df["taxes"] = df["taxes"]*(1-tax_rate_pct/100)
        df["insurance"] = df["insurance"]*(1-tax_rate_pct/100)
        df["maintenance"] =  df["maintenance"]*(1-tax_rate_pct/100)
        df["interest"] = df["interest"]*(1-tax_rate_pct/100)
        
#     display(df.style.bar(color='lightblue'))

    plot_payments(df[['taxes','insurance','maintenance','interest','principal','month']])

interactive(children=(IntSlider(value=30, continuous_update=False, description='Mortgage Years', max=30, min=1…